In [1]:
#imports

import pandas as pd

#from pg_utils_fn import create_block_mapped_dataset, fetch_block_mapping, populate_entity_mapping

In [2]:
import sqlite3


def create_block_mapped_dataset(dataset, mapping):
    """
    Create a mapped dataset by associating block codes with block names in the dataset.
    """
    dataset['block_name'] = dataset['block_name'].str.strip()
    dataset['block_code'] = dataset['block_name'].str.lower().map(mapping)
    dataset.loc[dataset['block_code'].isnull(), 'block_code'] = -2
    return dataset


def fetch_block_mapping():
    """
    Fetch the block mapping from the SQLite database.

    Returns:
    - A list of tuples containing the block entity name, LGD code, name variants, and parent entity.
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('lgd_database.db')
    cursor = conn.cursor()

    # Retrieve block data from the 'blocks' table
    cursor.execute("SELECT entityName, entityLGDCode, entityNameVariants, entityParent FROM block")
    data = cursor.fetchall()

    # Close the database connection
    conn.close()

    return data



def populate_entity_mapping_without_parent(data,column_name,parent_column_name):
    """
    Populates a entity mapping dictionary using data from a database and a local file.

    Returns:
        A defaultdict containing the mapping of entity names to their respective codes.
    """
    # Load unique entity data
    state_dataset = pd.read_csv('data.csv')
    unique_rows = state_dataset.drop_duplicates(subset=[column_name])
    unique_rows_lower = unique_rows.apply(lambda x: (x[column_name].strip().lower(), x[parent_column_name]), axis=1).tolist()

    entity_mapping = {}
    for entity_name, entity_code, entity_variants, parent_code in data:
        for row in unique_rows_lower:
            entity_name_lower = row[0]
            state_code = row[1]
            if entity_name_lower.strip() == entity_name.strip().lower():
                entity_mapping[entity_name_lower] = entity_code
                #print(entity_name_lower)
            else:
                if entity_variants:
                    for variant in entity_variants.split(','):
                        if variant.strip().lower() == entity_name_lower.strip():
                            entity_mapping[variant.strip().lower()] = entity_code
                            print(variant.strip().lower())

    return entity_mapping



In [3]:

block_dataset = pd.read_csv('data.csv')
data = fetch_block_mapping()
# Apply block mapping and create a new dataset
block_mapping = populate_entity_mapping_without_parent(data,'block_name','district_code')
mapped_dataset = create_block_mapped_dataset(block_dataset, block_mapping)
# Check if there are any unmatched names
unmatched_names = mapped_dataset[mapped_dataset['block_code'] == -2]['block_name']

KeyError: 'district_code'

In [ ]:


import pandas as pd

# Read the spreadsheet into a DataFrame
df = pd.read_csv('unmapped_subset.csv')

# Group the data by 'Block' and check for multiple 'Districts'
block_counts = df.groupby('block_name')['district_name'].nunique()

# Get the Blocks with multiple Districts
blocks_with_multiple_districts = block_counts[block_counts > 1].index.tolist()

result_dict = {}
if len(blocks_with_multiple_districts) > 0:
    for block in blocks_with_multiple_districts:
        districts = ', '.join(df[df['block_name'] == block]['district_name'].unique())
        result_dict[block] = districts

# Create a DataFrame from the result_dict
result_df = pd.DataFrame(result_dict.items(), columns=['Block', 'Districts'])

# Save the resulting DataFrame to a CSV file
result_df.to_csv('blocks_with_multiple_districts.csv', index=False)

# Read the two Excel files
df2 = pd.read_csv('blocks_with_multiple_districts.csv')
df1 = pd.read_csv('data.csv')

df1['block_name'] = df1['block_name'].str.lower()
df2['block_name'] = df2['block_name'].str.lower()

common_values = df1[~df1['block_name'].isin(df2['block_name'])]
#merged = df1.merge(df2, on='block_name', how='left', indicator=True)
#df1_unique = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

common_values.to_csv('output_sheet.csv', index=False)